# Libraries

In [ ]:
!pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 57.2 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import numpy as np
import pandas as pd
import scipy as sc
import openpyxl
import codecs
import copy
import os
import time
from datetime import datetime
from contextlib import closing
import sqlite3
from scipy.interpolate import interp1d
from contextlib import closing


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **Data parsing**

In [ ]:
# Подключение Google Drive
from google.colab import drive # Тут необходимо подлючится к диску через Google аккаунт
drive.mount('/content/drive', force_remount=False) # В одной сессии хватит подключится один раз, дальше подлючение не требуется (только при перезапуске сессии)

Mounted at /content/drive


In [ ]:
# Настройка путей
db_name = 'Data Archive.db'
local_db_path = '/content/' + db_name
drive_db_path = '/content/drive/MyDrive/Data Base/' + db_name  # Путь к базе данных в Google Drive
# excel_path = '/content/drive/My Drive/Data Base/Data Archive.xlsm'  # Путь к Excel файлу

In [ ]:
# Копируем базу из Drive если она существует
if os.path.exists(drive_db_path):
  !cp "{drive_db_path}" "{local_db_path}"
  print("База данных загружена из Google Drive")
else:
  print(f"База данных {db_name} отсутствует в Google Drive")

База данных загружена из Google Drive


In [ ]:
con = sqlite3.connect('Data Archive.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nTables:")
print("-------")
for table in tables:
    print(table[0])


for table in tables:
    table_name = table[0]
    print(f"\nТаблица: {table_name}")
    df = pd.read_sql(f"SELECT * FROM {table_name}", con)
    print(df)



# fx_curve = pd.read_sql("SELECT DISTINCT source FROM 'fx_rate'", con)
# print(fx_curve)


con.close()


Tables:
-------
curve_quote
fx_rate
swap_points_fwd

Таблица: curve_quote
                        date           curve   source currency currency_csa  \
0        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
1        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
2        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
3        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
4        2023-07-01 00:00:00   RUB_RUSFAR_3M     MOEX      RUB          RUB   
...                      ...             ...      ...      ...          ...   
2670122  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670123  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670124  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670125  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign      JPY          JPY   
2670126  2025-07-31 00:00:00  JPY TONAR TKFX  Foreign   

# **Архитектура DataArchive по таблицам (доступные источники, курсы)**

**fx_rate**

источники и доступные столбцы в них:

1.   Cbonds - px_last
2.   CBR - px_clir
3.   Foreign - px_low, px_high, px_close
4.   NCC - px_last
5.   OFFSHORE - в актуальной версии бд данных по источнику нет ни в одном из столбцов
6.   ONSHORE - px_clir
7.   RuData FOREX -  в актуальной версии бд данных по источнику нет ни в одном из столбцов
8.   RuData MOEX - px_low, px_high, px_last

**curve_quote**

источники:

1.   Cbonds - есть пропуски в данных
2.   Foreign - все данны есть
3.   RuData - тоже все ок

Для всех источников доступный столбец со значениями - zero_rate


Сейчас ставки подгружаются по фильтрам источника, валюты, тенора/дней, ставка не прописывается явно

**swap_points_fwd**

источники:

1.   Cbonds - last (единственное значение, не подходит для диапазона min/max)
2.   MOEX - low, high, last
3.   MOEX Backtrader - low, high

In [ ]:
# Если данные не парсятся и выдают ошибку по типу index out of list, то дописать в pd.to_datetime (value, format = '%Y-%m-%d')

# Reestr parsing

In [ ]:
reestr_initial = pd.read_excel('forward.xlsx')
reestr_initial.head()


,"№ сделки, п/п",№ сделки (идентификатор сделки),Статус сделки,"Дата сделки, дата пролонгации/изменения условий",Дата исполнения (расчетов),Контрагент,Тип инструмента,Покупаемая валюта,Сумма покупки,Продаваемая валюта,Сумма продажи,Курс сделки,"Признак продления, внесения изменений (да/нет)",Способ расчетов,"Справедливая стоимость ПФИ на текущую дату, в тыс.руб",Текущая дата на которую рассчитывалась справедливая стоимость ПФИ,Признак маржирования,Комментарии
0,1,NaN,новая,2024-12-27,2025-01-31,Х,форвард валютный,RUB,407160000,CNY,30000000,13.572,нет,NaN,1676,2024-12-31,NaN,NaN
1,2,NaN,новая,2024-12-30,2025-02-20,Х,форвард валютный,RUB,3830744,CNY,276588,13.850,нет,NaN,80.67,2024-12-31,NaN,NaN
2,3,NaN,новая,2024-09-25,2025-04-24,Х,форвард валютный,RUB,2304000000,CNY,180000000,12.800,нет,NaN,-,2024-12-31,NaN,NaN


# FUNCTIONS


In [ ]:
# Словарь приоритезации валют
currency_priority_dict = {
    'XDR': 0,
    'EUR': 1,
    'GBP': 2,
    'AUD': 3,
    'NZD': 4,
    'USD': 5,
    'CAD': 6,
    'CHF': 7,
    'TRY': 8,
    'DKK': 9,
    'NOK': 10,
    'ZAR': 11,
    'SEK': 12,
    'CNY': 13,
    'CNH': 14,
    'BRL': 15,
    'HKD': 16,
    'INR': 17,
    'CZK': 18,
    'KZT': 19,
    'JPY': 20,
    'BYN': 21,
    'SGD': 22,
    'KRW': 23,
    'HUF': 24,
    'RON': 25,
    'AZN': 26,
    'BGN': 27,
    'UZS': 28,
    'KGS': 29,
    'MDL': 30,
    'PLN': 31,
    'TMT': 32,
    'TJS': 33,
    'UAH': 34,
    'AMD': 35,
    'GEL': 36,
    'RUB': 37
}


# находим разницу по своп пунктам через ставки
# def swap_points_simple(spot, internal_rate, external_rate, time):
#   simple = (spot*(1+(internal_rate)*time/365)/(1+(external_rate)*time/365))-spot
#   return simple



In [ ]:
# Функция определения валютной пары
def currency_pair(curr1, curr2):

    try:
        if currency_priority_dict[curr1] < currency_priority_dict[curr2]:
            pair = curr1 + curr2
        else:
            pair = curr2 + curr1
        return pair
    except KeyError:
        print("Валюта не найдена в currency_priority_dict!")

In [ ]:
# Функция определения направления сделки
def deals_dir(curr1, curr2, curr_pair):
    """
    Input: currency 1, currency 2 and currency pair as str
    Output: direction of deal according to input parametres (SELL OR BUY)
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr1:
            ddir = 'SELL'
        elif curr_pair[:3] == curr2:
            ddir = 'BUY'
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        ddir = None
    return ddir

In [ ]:
# Функция определения суммы базовой валюты
def sum_curr_base(curr1, curr2, curr_pair, sum_sell, sum_buy):
    """
    Input: currency 1, currency 2 and currency pair as str; sum sell and sum buy
    Output: sum of base currency
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr1:
            sum_base = sum_sell
        elif curr_pair[:3] == curr2:
            sum_base = sum_buy
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        sum_base = None
    return sum_base

In [ ]:
# Функция определение суммы расчетной валюты
def sum_curr_calc(curr1, curr2, curr_pair, sum_sell, sum_buy):
    """
    Input: currency 1, currency 2 and currency pair as str; sum sell and sum buy
    Output: sum of calculation currency
    """
    if (curr1 in curr_pair) and (curr2 in curr_pair):
        if curr_pair[:3] == curr2:
            sum_curr = sum_sell
        elif curr_pair[:3] == curr1:
            sum_curr = sum_buy
    else:
        print('Несоответствие валют. Проверьте входные данные!')
        sum_curr = None
    return sum_curr

In [ ]:
# Функция определения расхождения спот курсов
def diff_spot(spot_rate, spot_min_rate, spot_max_rate, cb, ncc):
    """
    Input: spot_rate, spot_min_rate and spot_max_rate as float
    Output: spot max exchange rate on the spot_date from the specified source
    """
    if (spot_min_rate == None) or (spot_min_rate == 0) \
        or (spot_max_rate == None) or (spot_max_rate == 0):
        diff_spot = 'Нет данных'
    else:
        if (spot_rate >= spot_min_rate) and (spot_rate <= spot_max_rate) or (spot_rate == cb) or (spot_rate == ncc):
            diff_spot = 'В рынке'
        elif spot_rate < spot_min_rate:
            diff_spot = (spot_rate - spot_min_rate) / spot_min_rate
        else:
            diff_spot = (spot_rate - spot_max_rate) / spot_max_rate
    return diff_spot

In [ ]:
# Функция определения источника своп поинтов (напрямую с рынка для сделок < 3 дней или через ставки)
def type_swp(duration_of_contract):
  if duration_of_contract <= 3:
    type_swp = 'direct swp'
  else:
    type_swp = 'rates'
  return type_swp

In [ ]:
# Функция определения изменения форвардного курса

def forward_rate_change(days, rate_internal, rate_external):
    forward_change = ((1+(rate_internal/100))**(days/365))/((1+(rate_external/100))**(days/365))
    return forward_change


In [ ]:
# Функция определения расхождений форвардных курсов

def forward_rate_diff(fwd_bank, fwd_min, fwd_max):
    if (np.abs((fwd_min + fwd_max)/2-fwd_bank)) <= (((np.maximum(fwd_min, fwd_max)) - np.minimum(fwd_min, fwd_max))/2):
        fwd_diff = 'В рынке'
    else:
        if (fwd_bank < np.minimum(fwd_min, fwd_max)):
            fwd_diff = fwd_bank - np.minimum(fwd_min, fwd_max)
        else:
            fwd_diff = fwd_bank - np.maximum(fwd_min, fwd_max)
    return fwd_diff

In [ ]:
# определяем по сроку сделки больший тенор для ставок
def tenor1h(days_of_swap):

  if days_of_swap < 7:
    tenor = 1
  if days_of_swap <=7 and days_of_swap >=14:
    tenor = 7
  if days_of_swap <=14 and days_of_swap >=7:
    tenor = 14
  if days_of_swap <=30 and days_of_swap >=14:
    tenor = 31
  if days_of_swap <= 58 and days_of_swap >=31:
    tenor = 59
  if days_of_swap <= 90 and days_of_swap >=60:
    tenor = 90
  if days_of_swap <= 180 and days_of_swap >=90:
    tenor = 180
  if days_of_swap <= 270 and days_of_swap >=180:
    tenor = 270
  if days_of_swap <= 360 and days_of_swap >=270:
    tenor = 360
  return tenor

# определяем по сроку сделки меньший тенор для ставок
def tenor2l(days_of_swap):
  if days_of_swap < 7:
    tenor = 1
  if days_of_swap <=14 and days_of_swap >=7:
    tenor = 7
  if days_of_swap <=30 and days_of_swap >=14:
    tenor = 14
  if days_of_swap <= 60 and days_of_swap >=30:
    tenor = 31
  if days_of_swap <= 90 and days_of_swap >=60:
    tenor = 59
  if days_of_swap <= 180 and days_of_swap >=90:
    tenor = 90
  if days_of_swap <= 270 and days_of_swap >=180:
    tenor = 180
  if days_of_swap <= 360 and days_of_swap >= 270:
    tenor = 360
  return tenor

In [ ]:
# задаем кривые для валют
curves = {
    'CNY': 'CNY SHIBOR REF',
    'USD': 'USD SOFR REF',
    'EUR': 'EUR ESTR REF',
    'AUD': 'AUD AONIA ICAA',
    'CAD': 'CAD CORRA REF',
    'CHF': 'CHF SARON ICAP',
    'GBP': 'GBP SONIA TWMK',
    'HKD': 'HKD HONIA TRHK',
    'JPY': 'JPY TONAR TKFX',
    'RUB': 'RUB RUONIA COMB',
    'TRY': 'TRY TLREF REF'
    }


def curve_ident(currency_1):
        x = curves.get(currency_1)
        return x

In [ ]:
#Eли брать ставки с рефа, то там есть овернайт по CNY, USD, EUR

In [ ]:
# В текущей редакции этот блок НЕ ПРОГОНЯТЬ, пока стоит костыль из-за не совсем полных данных в DataArchive. Как будет отрегулирована база, этот код будет использоваться (уже протестирован на хороших данных)

#находим внутренюю ставку меньшую
def rate_domestic_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_l

#находим внутренюю ставку большую
def rate_domestic_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_h

#находим внешнюю ставку меньшую
def rate_foreign_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}'and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_l

#находим внешнюю ставку большую
def rate_foreign_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_h


In [ ]:
# определяем СС
def fair_value(cur_type, buy_amnt, sell_amnt, disc_internal, disc_external, spot):
    if cur_type == 'RUB':
        fv = (buy_amnt * 1 * disc_internal - sell_amnt * spot * disc_external)/1000
    else:
        fv = (buy_amnt * spot * disc_external - sell_amnt * 1 * disc_internal)/1000
    return fv

In [ ]:
# функция определения расхождения от суммы сделки
def dif_from_sum(abs, sum):
    if abs == 'В рынке':
        dfs = '0%'
    else:
        dfs = (abs/(sum/1000))
    return dfs

# CALCULATE REESTR

In [ ]:
reestr_initial = pd.read_excel('forward.xlsx')

In [ ]:
# Загружаем реестр
reestr_mini = pd.read_excel('forward.xlsx')

In [ ]:
# Определяем дату сделки и дату оценки
reestr_mini['Дата сделки'] = reestr_mini['Дата сделки, дата пролонгации/изменения условий']
reestr_mini['Дата оценки'] = reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ']


In [ ]:
# Блок с определением валют
reestr_mini['Продаваемая валюта'] = reestr_mini['Продаваемая валюта']
reestr_mini['Покупаемая валюта'] = reestr_mini['Покупаемая валюта']
reestr_mini['Валюта расчетов'] = input("Введите валюту расчетов (например: RUB): ")

# Вспомогательная часть для валютных пар
reestr_mini['Валютная пара_1'] = reestr_mini['Покупаемая валюта'] + reestr_mini['Валюта расчетов']
reestr_mini['Валютная пара_2'] = reestr_mini['Продаваемая валюта'] + reestr_mini['Валюта расчетов']

# Валютная пара
curr_pair = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    curr_pair.append(currency_pair(reestr_mini['Продаваемая валюта'].iloc[i], reestr_mini['Покупаемая валюта'].iloc[i]))
reestr_mini['Валютная пара'] = curr_pair


Введите валюту расчетов (например: RUB): RUB


In [ ]:
# Направление сделки
direction = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    direction.append(deals_dir(reestr_mini['Продаваемая валюта'].iloc[i], reestr_mini['Покупаемая валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i]))
reestr_mini['Направление сделки'] = direction
reestr_mini['Название'] = reestr_mini['Валютная пара'] + reestr_mini['Направление сделки']


In [ ]:
# Сумма в базовой валюте
sum_base = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    sum_base.append(sum_curr_base(reestr_mini['Продаваемая валюта'].iloc[i], reestr_mini['Покупаемая валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i], reestr_mini['Сумма продажи'].iloc[i], reestr_mini['Сумма покупки'].iloc[i]))
reestr_mini['Сумма в базовой валюте'] = sum_base


In [ ]:
# Сумма в расчетной валюте
sum_calc = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    sum_calc.append(sum_curr_calc(reestr_mini['Продаваемая валюта'].iloc[i], reestr_mini['Покупаемая валюта'].iloc[i], reestr_mini['Валютная пара'].iloc[i], reestr_mini['Сумма продажи'].iloc[i], reestr_mini['Сумма покупки'].iloc[i]))
reestr_mini['Сумма в расчетной валюте'] = sum_calc


In [ ]:
# Определяем срок сделки (дата исполнения - дата расчетов, для вспомогательной таблицы при оценке СС и для оценки от даты сделки непосредственно до даты оценки)
days = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    a = pd.to_datetime(reestr_mini['Дата исполнения (расчетов)'].iloc[i], dayfirst=True) - pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/изменения условий'].iloc[i], dayfirst=True)
    days.append(a.days)
reestr_mini['Срок'] = days


In [ ]:
# Спот мин на дату сделки
spot_min = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    table_name = 'fx_rate'
    source = 'RuData MOEX'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i] + "_TOM"
    date = pd.to_datetime(reestr_mini['Дата сделки'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_low from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_min.append(result[0][0] if result else None)
reestr_mini['Спот мин сделка'] = spot_min


In [ ]:
# Спот макс на дату сделки
spot_max_val_date = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    table_name = 'fx_rate'
    source = 'RuData MOEX'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i] + "_TOM"
    date = pd.to_datetime(reestr_mini['Дата сделки'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_high from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_max_val_date.append(result[0][0] if result else None)
reestr_mini['Спот макс сделка'] = spot_max_val_date


In [ ]:
# Идентификаторы кривых ставок
curve_id_1 = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    curve_id_1.append(curve_ident(reestr_mini['Продаваемая валюта'].iloc[i]))
reestr_mini['название ставки в расчетной валюте'] = curve_id_1

curve_id_2 = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    curve_id_2.append(curve_ident(reestr_mini['Покупаемая валюта'].iloc[i]))
reestr_mini['название ставки в базовой валюте'] = curve_id_2


In [ ]:
# Определяем тенора для парсинга ставок
tenor_low =[]
for i in range(len(reestr_mini['№ сделки, п/п'])):
    tenor_low.append(tenor2l(reestr_mini['Срок'][i]))
reestr_mini['Тенор мин'] = tenor_low

tenor_high =[]
for i in range(len(reestr_mini['№ сделки, п/п'])):
    tenor_high.append(tenor1h(reestr_mini['Срок'][i]))
reestr_mini['Тенор макс'] = tenor_high


In [ ]:
reestr_mini

,"№ сделки, п/п",№ сделки (идентификатор сделки),Статус сделки,"Дата сделки, дата пролонгации/изменения условий",Дата исполнения (расчетов),Контрагент,Тип инструмента,Покупаемая валюта,Сумма покупки,Продаваемая валюта,...,Название,Сумма в базовой валюте,Сумма в расчетной валюте,Срок,Спот мин сделка,Спот макс сделка,название ставки в расчетной валюте,название ставки в базовой валюте,Тенор мин,Тенор макс
0,1,NaN,новая,2024-12-27,2025-01-31,Х,форвард валютный,RUB,407160000,CNY,...,CNYRUBSELL,30000000,407160000,35,13.3705,13.5385,CNY SHIBOR REF,RUB RUONIA COMB,31,59
1,2,NaN,новая,2024-12-30,2025-02-20,Х,форвард валютный,RUB,3830744,CNY,...,CNYRUBSELL,276588,3830744,52,13.5020,13.8125,CNY SHIBOR REF,RUB RUONIA COMB,31,59
2,3,NaN,новая,2024-09-25,2025-04-24,Х,форвард валютный,RUB,2304000000,CNY,...,CNYRUBSELL,180000000,2304000000,211,13.0575,13.1950,CNY SHIBOR REF,RUB RUONIA COMB,180,270


In [ ]:
# Здесь определяются ставки на датау сделки, данный блок будет преобразовываться с появлением алгоритма проверки данных (функции в блок функций будут перенесены, а сами расчетные строки преобразованы в более лаконичный вид)

#  Функции в дальнейшем будут перемещены в блок функций, пока находятся в самом коде для удобства пофиксить источники данных и тд

# находим внутреннюю ставку меньшую
def rate_domestic_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_l

rates_domestic_l = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_domestic_l.append(rate_domestic_pars_l(reestr_mini['Покупаемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/изменения условий'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['название ставки в базовой валюте'].iloc[i]))
reestr_mini['Ставка внутреняя мин дата сделки'] = rates_domestic_l

#находим внутренюю ставку большую
def rate_domestic_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_h

rates_domestic_h = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_domestic_h.append(rate_domestic_pars_h(reestr_mini['Покупаемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/изменения условий'].iloc[i]), reestr_mini['Тенор макс'].iloc[i], reestr_mini['название ставки в базовой валюте'].iloc[i]))
reestr_mini['Ставка внутреняя макс дата сделки'] = rates_domestic_h

#находим внешнюю ставку меньшую
def rate_foreign_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}'and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_l

rates_foreign_l = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_foreign_l.append(rate_foreign_pars_l(reestr_mini['Продаваемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/изменения условий'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['название ставки в расчетной валюте'].iloc[i]))
reestr_mini['Ставка внешняя мин дата сделки'] = rates_foreign_l

#находим внешнюю ставку большую
def rate_foreign_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_h

# стоит костыль для тенора, так как в БД данные с рефа не все, на 60 дней данных нет, в дальнейшем убрать и чделать по аналогии с внутренней валютой. Такой вариант используется реализации кода и общего тестирования расчетных величин
tenor_max = "90"
rates_foreign_h = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_foreign_h.append(rate_foreign_pars_h(reestr_mini['Продаваемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Дата сделки, дата пролонгации/изменения условий'].iloc[i]), tenor_max, reestr_mini['название ставки в расчетной валюте'].iloc[i]))
reestr_mini['Ставка внешняя макс дата сделки'] = rates_foreign_h


IndexError: list index out of range

In [ ]:
# Интерполяция ставок
reestr_mini['Интер ставка внутреняя от даты сделки до экспирации'] = reestr_mini.apply(lambda row: np.interp(row['Срок'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внутреняя мин дата сделки'], row['Ставка внутреняя макс дата сделки']]), axis=1)
reestr_mini['Интер ставка внешняя от даты сделки до экспирации'] = reestr_mini.apply(lambda row: np.interp(row['Срок'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внешняя мин дата сделки'], row['Ставка внешняя макс дата сделки']]), axis=1)


In [ ]:
# Изменение форвардного курса
change_of_fwd_rate = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    change_of_fwd_rate.append(forward_rate_change(reestr_mini['Срок'].iloc[i], reestr_mini['Интер ставка внутреняя от даты сделки до экспирации'].iloc[i], reestr_mini['Интер ставка внешняя от даты сделки до экспирации'].iloc[i]))
reestr_mini['Изменение форвардного курса'] = change_of_fwd_rate


In [ ]:
#Определяем мин-макс форвардного курса
reestr_mini['Форвардный курс мин'] = reestr_mini.apply(lambda row: row['Спот мин сделка'] * row['Изменение форвардного курса'], axis =1)
reestr_mini['Форвардный курс макс'] = reestr_mini.apply(lambda row: row['Спот макс сделка'] * row['Изменение форвардного курса'], axis =1)


In [ ]:
#Расхождение в курсах, abs
abs_diff = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    abs_diff.append(forward_rate_diff(reestr_mini['Курс сделки'].iloc[i], reestr_mini['Форвардный курс мин'].iloc[i], reestr_mini['Форвардный курс макс'].iloc[i]))
reestr_mini['Расхождения, abs'] = abs_diff


In [ ]:
reestr_mini

,"№ сделки, п/п",№ сделки (идентификатор сделки),Статус сделки,"Дата сделки, дата пролонгации/изменения условий",Дата исполнения (расчетов),Контрагент,Тип инструмента,Покупаемая валюта,Сумма покупки,Продаваемая валюта,...,Интер ставка внутреняя от даты сделки до даты оценки,Интер ставка внешняя от даты сделки до даты оценки,Дисконт-фактор для внешней валюты,Дисконт-фактор для внутренней валюты,Справедливая стоимость Банка,Справедливая стоимость мин,Справедливая стоимость макс,Расхождение с СС Банка (абс),Расхождение с СС Банка (%),Расхождение от суммы сделки
0,1,NaN,новая,2024-12-27,2025-01-31,Х,форвард валютный,RUB,407160000,CNY,...,15.150000,1.681000,0.998574,0.987296,1676.00,-2494.944563,-2531.392396,4207.392396,0.010334,0.010334
1,2,NaN,новая,2024-12-30,2025-02-20,Х,форвард валютный,RUB,3830744,CNY,...,15.221429,1.684214,0.997652,0.979175,80.67,25.243827,80.939262,В рынке,0.000000,0%


In [ ]:
#Расхождения в % по курсам форвардным
percent_change = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    try:
        percent_change.append(reestr_mini['Расхождения, asb'][i] / reestr_mini['Форвардный курс макс'][i])
    except TypeError:
        percent_change.append('0 %')
reestr_mini['Расхождения %'] =  percent_change


In [ ]:
# Спот мин на дату оценки
spot_min_value = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    table_name = 'fx_rate'
    source = 'RuData MOEX'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i] + "_TOM"
    date = pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_low from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_min_value.append(result[0][0] if result else None)
reestr_mini['Спот мин оценка'] = spot_min_value


In [ ]:
# Спот макс на дату сделки
spot_max_val = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    table_name = 'fx_rate'
    source = 'RuData MOEX'
    pair_spot_name = reestr_mini['Валютная пара'].iloc[i] + "_TOM"
    date = pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    with closing(sqlite3.connect('Data Archive.db')) as connection:
        result = connection.cursor().execute(f"""select px_high from '{table_name}' where source = '{source}' and pair = '{pair_spot_name}' and date = '{date}' """).fetchall()
        spot_max_val.append(result[0][0] if result else None)
reestr_mini['Спот макс оценка'] = spot_max_val


In [ ]:
#Срок дата расчетов - дата оценки
days_1 = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    a_1 = pd.to_datetime(reestr_mini['Дата исполнения (расчетов)'].iloc[i], dayfirst=True) - pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i], dayfirst=True)
    days_1.append(a_1.days)
reestr_mini['Срок от даты расчетов до даты оценки'] = days_1


In [ ]:
# Здесь ставки на дату оценки. Функции в дальнейшем будут перемещены в блок функций, пока находятся в самом коде для удобства пофиксить источники данных и тд

# находим внутреннюю ставку меньшую
def rate_domestic_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_l

rates_domestic_l_v = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_domestic_l_v.append(rate_domestic_pars_l(reestr_mini['Покупаемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['название ставки в базовой валюте'].iloc[i]))
reestr_mini['Ставка внутреняя мин дата оценки'] = rates_domestic_l_v

#находим внутренюю ставку большую
def rate_domestic_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_d_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_d_h

rates_domestic_h_v = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_domestic_h_v.append(rate_domestic_pars_h(reestr_mini['Покупаемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i]), reestr_mini['Тенор макс'].iloc[i], reestr_mini['название ставки в базовой валюте'].iloc[i]))
reestr_mini['Ставка внутреняя макс дата оценки'] = rates_domestic_h_v


# #находим внешнюю ставку меньшую
def rate_foreign_pars_l(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_l = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}'and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_l

rates_foreign_l_v = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_foreign_l_v.append(rate_foreign_pars_l(reestr_mini['Продаваемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i]), reestr_mini['Тенор мин'].iloc[i], reestr_mini['название ставки в расчетной валюте'].iloc[i]))
reestr_mini['Ставка внешняя мин дата оценки'] = rates_foreign_l_v

# #находим внешнюю ставку большую
def rate_foreign_pars_h(currency, date, days, curve):
  with closing(sqlite3.connect('Data Archive.db')) as connection:
    table_name = 'curve_quote'
    source = 'Foreign'
    #source = input("print desired source of info (Cbonds, Foreign, MARKIT, MOEX, RuData): ")
    rate_f_h = ((connection.cursor().execute(f"""select zero_rate from '{table_name}' where currency = '{currency}' and source = '{source}' and date = '{date}' and days = '{days}' and curve = '{curve}' """)).fetchall()[0][0])
  return rate_f_h

# стоит костыль для тенора, так как в БД данные с рефа не все, на 60 дней данных нет, в дальнейшем убрать и делать по аналогии с внутренней валютой. Такой вариант используется реализации кода и общего тестирования расчетных величин
tenor_max = "90"
rates_foreign_h = []
for i in range(len(reestr_mini['№ сделки, п/п'])):
    rates_foreign_h.append(rate_foreign_pars_h(reestr_mini['Продаваемая валюта'].iloc[i], pd.to_datetime(reestr_mini['Текущая дата на которую рассчитывалась справедливая стоимость ПФИ'].iloc[i]), tenor_max, reestr_mini['название ставки в расчетной валюте'].iloc[i]))
reestr_mini['Ставка внешняя макс дата оценки'] = rates_foreign_h


In [ ]:
# Интерполяция ставок
reestr_mini['Интер ставка внутреняя от даты сделки до даты оценки'] = reestr_mini.apply(lambda row: np.interp(row['Срок от даты расчетов до даты оценки'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внутреняя мин дата оценки'], row['Ставка внутреняя макс дата оценки']]), axis=1)
reestr_mini['Интер ставка внешняя от даты сделки до даты оценки'] = reestr_mini.apply(lambda row: np.interp(row['Срок от даты расчетов до даты оценки'], [row['Тенор мин'], row['Тенор макс']], [row['Ставка внешняя мин дата оценки'], row['Ставка внешняя макс дата оценки']]), axis=1)


In [ ]:
# Определяем дисконт-факторы для базовой и расчетной валют (1 тайп)
# reestr_mini['Дисконт-фактор для внешней валюты'] = reestr_mini.apply(lambda row: (1 + row['Интер ставка внешняя от даты сделки до даты оценки'])**-(row['Срок от даты расчетов до даты оценки']/365), axis=1)
# reestr_mini['Дисконт-фактор для внутренней валюты'] = reestr_mini.apply(lambda row: (1 + row['Интер ставка внутреняя от даты сделки до даты оценки'])**-(row['Срок от даты расчетов до даты оценки']/365), axis=1)

# 2 тайп более универсальный
reestr_mini['Дисконт-фактор для внешней валюты'] = reestr_mini.apply(lambda row: (1/ (1+row['Интер ставка внешняя от даты сделки до даты оценки']/100*row['Срок от даты расчетов до даты оценки']/365)), axis=1)
reestr_mini['Дисконт-фактор для внутренней валюты'] = reestr_mini.apply(lambda row: (1/ (1+row['Интер ставка внутреняя от даты сделки до даты оценки']/100*row['Срок от даты расчетов до даты оценки']/365)), axis =1)


In [ ]:
# СС часть (Банк + Служба)
reestr_mini['Справедливая стоимость Банка'] = reestr_mini['Справедливая стоимость ПФИ на текущую дату, в тыс.руб']

# FV by min spot
fvl =[]
for i in range(len(reestr_mini['№ сделки, п/п'])):
    fvl.append(fair_value(reestr_mini['Покупаемая валюта'].iloc[i], reestr_mini['Сумма покупки'].iloc[i], reestr_mini['Сумма продажи'].iloc[i], reestr_mini['Дисконт-фактор для внутренней валюты'].iloc[i], reestr_mini['Дисконт-фактор для внешней валюты'][i], reestr_mini['Спот мин оценка'].iloc[i]))
reestr_mini['Справедливая стоимость мин'] = fvl

# FV by max spot
fvh =[]
for i in range(len(reestr_mini['№ сделки, п/п'])):
    fvh.append(fair_value(reestr_mini['Покупаемая валюта'].iloc[i], reestr_mini['Сумма покупки'].iloc[i], reestr_mini['Сумма продажи'].iloc[i], reestr_mini['Дисконт-фактор для внешней валюты'].iloc[i], reestr_mini['Дисконт-фактор для внутренней валюты'][i], reestr_mini['Спот макс оценка'].iloc[i]))
reestr_mini['Справедливая стоимость макс'] = fvh


In [ ]:
# Расхождение с СС Банка
fv_delta_abs_1 = []
fv_delta_percent_1 = []

for i in range(len(reestr_mini['№ сделки, п/п'])):
    bank_fv = reestr_mini['Справедливая стоимость Банка'].iloc[i]
    low_fv = reestr_mini['Справедливая стоимость мин'].iloc[i]
    high_fv = reestr_mini['Справедливая стоимость макс'].iloc[i]

    if bank_fv < low_fv:
        fv_delta_abs_1.append(bank_fv - low_fv)
        fv_delta_percent_1.append((bank_fv - low_fv) * 1000 / reestr_mini['Сумма в расчетной валюте'].iloc[i])
    elif bank_fv > high_fv:
        fv_delta_abs_1.append(bank_fv - high_fv)
        fv_delta_percent_1.append((bank_fv - high_fv) * 1000 / reestr_mini['Сумма в расчетной валюте'].iloc[i])
    else:
        fv_delta_abs_1.append("В рынке")
        fv_delta_percent_1.append(0.)

reestr_mini['Расхождение с СС Банка (абс)'] = fv_delta_abs_1
reestr_mini['Расхождение с СС Банка (%)'] = fv_delta_percent_1


In [ ]:
# Расхождение от суммы сделки
diff_from_sum =[]
for i in range(len(reestr_mini['№ сделки, п/п'])):
    diff_from_sum.append(dif_from_sum((reestr_mini['Расхождение с СС Банка (абс)'].iloc[i]), (reestr_mini['Сумма в расчетной валюте'].iloc[i])))
reestr_mini['Расхождение от суммы сделки'] = diff_from_sum

reestr_mini

,"№ сделки, п/п",№ сделки (идентификатор сделки),Статус сделки,"Дата сделки, дата пролонгации/изменения условий",Дата исполнения (расчетов),Контрагент,Тип инструмента,Покупаемая валюта,Сумма покупки,Продаваемая валюта,...,Интер ставка внутреняя от даты сделки до даты оценки,Интер ставка внешняя от даты сделки до даты оценки,Дисконт-фактор для внешней валюты,Дисконт-фактор для внутренней валюты,Справедливая стоимость Банка,Справедливая стоимость мин,Справедливая стоимость макс,Расхождение с СС Банка (абс),Расхождение с СС Банка (%),Расхождение от суммы сделки
0,1,NaN,новая,2024-12-27,2025-01-31,Х,форвард валютный,RUB,407160000,CNY,...,15.150000,1.681000,0.998574,0.987296,1676.00,-2494.944563,-2531.392396,4207.392396,0.010334,0.010334
1,2,NaN,новая,2024-12-30,2025-02-20,Х,форвард валютный,RUB,3830744,CNY,...,15.221429,1.684214,0.997652,0.979175,80.67,25.243827,80.939262,В рынке,0.000000,0%


In [ ]:
reestr_mini.to_excel('full_calculations_of_reestr.xlsx')

# **Create final reestr**

In [ ]:
reestr_initial = pd.read_excel('forward.xlsx')

In [ ]:
# Заполняем первоначальный реестр расчетными полями

reestr_initial['Валютная пара'] = reestr_mini['Валютная пара']
reestr_initial['Направление сделки'] = reestr_mini['Направление сделки']
reestr_initial['Срок'] = reestr_mini['Срок']
reestr_initial['Курс сделки (форвард)'] = reestr_mini['Курс сделки']
reestr_initial['Форвардный курс min'] = reestr_mini['Форвардный курс мин']
reestr_initial['Форвардный курс max'] = reestr_mini['Форвардный курс макс']
reestr_initial['Расхождение, %'] = reestr_mini['Расхождения %']
reestr_initial['Расхождение abs'] = reestr_mini['Расхождения, abs']
reestr_initial['Дней до экспирации'] = reestr_mini['Срок от даты расчетов до даты оценки'] # в расчетах используются 2 срока: от даты сделки до даты расчетов и от даты расчетов до даты оценки СС (для удобства они разделены и отображаются в разных частях реестра)
reestr_initial['Справедливая стоимость Банка, тыс.руб'] = reestr_mini['Справедливая стоимость Банка']
reestr_initial['Справедливая стоимость Службы min, тыс. руб'] = reestr_mini['Справедливая стоимость мин']
reestr_initial['Справедливая стоимость Службы max, тыс. руб'] = reestr_mini['Справедливая стоимость макс']
reestr_initial['Расхождение СС abs'] = reestr_mini['Расхождение с СС Банка (абс)']
reestr_initial['Расхождение СС %'] = reestr_mini['Расхождение с СС Банка (%)']
reestr_initial['Расхождение СС от суммы сделки'] = reestr_mini['Расхождение от суммы сделки']

reestr_initial.head()

,"№ сделки, п/п",№ сделки (идентификатор сделки),Статус сделки,"Дата сделки, дата пролонгации/изменения условий",Дата исполнения (расчетов),Контрагент,Тип инструмента,Покупаемая валюта,Сумма покупки,Продаваемая валюта,...,Расхождение abs,Дней до экспирации,"Справедливая стоимость Банка, тыс.руб","Справедливая стоимость Службы min, тыс. руб","Справедливая стоимость Службы max, тыс. руб",Расхождение %,Расхождение от суммы сделки,Расхождение СС abs,Расхождение СС %,Расхождение СС от суммы сделки
0,1,NaN,новая,2024-12-27,2025-01-31,Х,форвард валютный,RUB,407160000,CNY,...,В рынке,31,1676.00,-2494.944563,-2531.392396,0.010334,0.010334,4207.392396,0.010334,0.010334
1,2,NaN,новая,2024-12-30,2025-02-20,Х,форвард валютный,RUB,3830744,CNY,...,В рынке,51,80.67,25.243827,80.939262,0.000000,0%,В рынке,0.000000,0%


In [ ]:
#Скачиваем финальный реестр

reestr_initial.to_excel('reestr_final.xlsx')

# **Design**

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.1 MB/s eta 0:00:00


In [ ]:
import xlsxwriter
import datetime

with pd.ExcelWriter('final_forward_with_design.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'nan_inf_to_errors': True}}) as wb:

    data_border_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter'
    })

    date_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'num_format': 'dd.mm.yyyy'
    })


    number_format = wb.book.add_format({
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'num_format': '### ### ###'
    })

    reestr_initial.to_excel(wb, sheet_name='СПОТ', startrow=7, startcol=1, index=False)

    sheet = wb.sheets['СПОТ']

    sheet.hide_gridlines(2)

    title_format = wb.book.add_format({
        'bold' : True,
        'align': 'left',
        'valign': 'vcenter'})

    sheet.write(6, 1, "Валютный форвард", title_format)
    sheet.write(2, 17, "Для служебного пользования")
    sheet.write(3, 17, "Приложение 3 к письму")

    num_rows = len(reestr_initial)
    num_cols = len(reestr_initial.columns)

    number_columns = [7, 8, 9, 11, 19, 20]
    date_columns = [4, 5, 16]


    for row in range(8, 8 + num_rows):
        for col in range(1, 1 + num_cols):
            value = reestr_initial.iloc[row-8, col-1]


            if col in date_columns:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", date_format)
                else:

                    if isinstance(value, (pd.Timestamp, datetime.datetime)):
                        sheet.write_datetime(row, col, value, date_format)
                    else:
                        sheet.write(row, col, value, date_format)


            elif col in number_columns:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", number_format)
                else:

                    if isinstance(value, (int, float)) and not pd.isna(value):
                        sheet.write_number(row, col, value, number_format)
                    else:
                        sheet.write(row, col, value, number_format)

            else:

                if pd.isna(value) or value in [float('inf'), float('-inf')]:
                    sheet.write(row, col, "", data_border_format)
                else:
                    sheet.write(row, col, value, data_border_format)


    gray_format = wb.book.add_format({
        'bold': True,
        'font_color': '#000000',
        'bg_color': '#DCDCDC',  ##DCDCDC, c0c0c0 можно сделать еще посветлее серый
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})

    blue_format = wb.book.add_format({
        'bold': True,
        'font_color': '#FFFFFF',
        'bg_color': '#0066cc',
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})



    black_format = wb.book.add_format({
        'bold': True,
        'font_color': '#FFFFFF',
        'bg_color': '#404040',
        'align': 'center',
        'valign': 'vcenter',
        'border': 1})



    for col_idx, col_name in enumerate(reestr_initial.columns):
        if col_idx < 20:
            sheet.write(7, col_idx + 1, col_name, gray_format)
        elif col_idx < 27:
            sheet.write(7, col_idx + 1, col_name, blue_format)
        elif col_idx < 32:
            sheet.write(7, col_idx + 1, col_name, black_format)
        # else:
        #     sheet.write(7, col_idx + 1, col_name, gray_format)


    for i, col in enumerate(reestr_initial.columns):
        max_len = 0
        for row in reestr_initial[col]:
            try:

                if pd.isna(row) or row in [float('inf'), float('-inf')]:
                    cell_value = ""
                else:

                    if i in [2, 3] and isinstance(row, (pd.Timestamp, datetime.datetime)):
                        cell_value = row.strftime('%d.%m.%Y')
                    else:
                        cell_value = str(row)
                max_len = max(max_len, len(cell_value))
            except:
                pass
        max_len = max(max_len, len(col))
        sheet.set_column(i + 1, i + 1, max_len + 2)

    for i in range(len(reestr_initial) + 1):
        sheet.set_row(i + 7, None)